In [90]:
!pip install kaggle

In [44]:
# Extracting the compressed data set

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [91]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [92]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [93]:
#loading the data from the csv file to pandas dataframe
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = "ISO-8859-1")
data.shape
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [95]:
#naming the coloumns
data_col = ["target","id",'date',"flag","user","text"]
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = data_col,encoding = "ISO-8859-1")
data.shape

(1048576, 6)

In [96]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [97]:
data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [98]:
#Checking the distribution of the target Coloumn

data['target'].value_counts()

0    800000
4    248576
Name: target, dtype: int64

Converting the target "4" to "1"   Preprocessing

In [99]:
data.replace({'target':{4:1}}, inplace = True)

In [100]:
data['target'].value_counts()

0    800000
1    248576
Name: target, dtype: int64

0 -- negative tweet

1--positive tweet

**Stemming** - it is a process of reducing a word to its root word (porter stemmer module is used)

Example: actor, actress, acting these words to a root word to **act**

In [101]:
port_stem = PorterStemmer()

In [102]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content) # matching of symbols that doesn't come under (a-zA-Z) will be removed
  stemmed_content = stemmed_content.lower()
  stemmed_content =stemmed_content.split()
  #PortStemmed function is used
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # useless are words are removed from the list
  stemmed_content = ' '.join(stemmed_content) # the letters are joined up

  return stemmed_content #contains only the letters & words and not the special charecters or such

In [103]:
data['stemmed_content'] = data['text'].apply(stemming)

In [104]:
data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


*ignore the raw data & take up the trained data*

In [106]:
print(data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1048571                              grandma make dinenr mum
1048572             mid morn snack time bowl chees noodl yum
1048573         shadela say like termini movi come like word
1048574                      destinyhop im great thaank wbuu
1048575                           cant wait til date weekend
Name: stemmed_content, Length: 1048576, dtype: object


In [107]:
print(data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1048571    1
1048572    1
1048573    1
1048574    1
1048575    1
Name: target, Length: 1048576, dtype: int64


Diffrentiating a positive tweet & a negative tweet through key words

In [108]:
#Seperating the data & label
X = data['stemmed_content'].values
Y = data['target'].values

In [109]:
print(X)
print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'shadela say like termini movi come like word'
 'destinyhop im great thaank wbuu' 'cant wait til date weekend']
[0 0 0 ... 1 1 1]


Splitting the data training data & test data

In [110]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y, random_state = 2)

**Stratify** - means equal distribution of the 2 classes

In [111]:
print(X.shape,X_train.shape,X_test.shape)
print(Y.shape, Y_train.shape,Y_test.shape)

(1048576,) (786432,) (262144,)
(1048576,) (786432,) (262144,)


Converting the textual data to numerical data - using **Vectorizer**

Encoding texual data to numeric data

Transforming only the training data

In [112]:
vector = TfidfVectorizer()
X_train = vector.fit_transform(X_train)
X_test = vector.transform(X_test)

In [113]:
print(X_train)
print(X_test)

  (0, 253024)	0.16038610274510973
  (0, 103755)	0.20457286497720015
  (0, 55064)	0.26244992759191677
  (0, 61989)	0.38286402557724253
  (0, 103661)	0.14210491399258973
  (0, 10353)	0.15580144550172695
  (0, 263013)	0.2617403360905449
  (0, 224168)	0.2672852036082925
  (0, 263240)	0.35556267451657175
  (0, 211059)	0.2854890548955961
  (0, 189843)	0.15887296710420207
  (0, 280657)	0.1373658931165813
  (0, 255804)	0.43393779116901565
  (0, 282557)	0.235612262761079
  (0, 72644)	0.19154676862387335
  (1, 26872)	0.29155272894893064
  (1, 94206)	0.561407958365684
  (1, 48133)	0.4076536788812564
  (1, 129867)	0.47485263214586204
  (1, 164815)	0.21636950222301105
  (1, 260121)	0.4016662854567163
  (2, 85730)	0.3611882584605015
  (2, 125705)	0.8387823752925426
  (2, 210545)	0.4074152290340531
  (3, 216385)	0.3396811824438364
  :	:
  (786428, 197947)	0.2161819500463266
  (786428, 299446)	0.2181182060426654
  (786428, 117076)	0.34963819271150004
  (786428, 111944)	0.2640637259937648
  (786428, 15

**Training our logistic regression  model**

In [114]:
model = LogisticRegression(max_iter = 1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

*Model Evaluation*

**Acccuracy Score**

In [115]:
# Accuracy score on the training data

X_train_predict = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_predict)

In [116]:
print('Accuracy Score:', training_data_accuracy)

Accuracy Score: 0.8575146993001302


In [117]:
# Accuracy score on the test data

X_test_predict = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_predict)

In [118]:
print('Accuracy Score:', test_data_accuracy)

Accuracy Score: 0.8339118957519531


**Saving the trained model**

In [119]:
import pickle
filename = 'SAT_data.sav'
pickle.dump(model, open(filename, 'wb'))


**Using the saved model for future predictions**

In [120]:
# Loading the saved model
lmodel = pickle.load(open('/content/SAT_data.sav','rb'))

In [121]:
newX_data = X_test[200]
print(Y_test[200])

predict = lmodel.predict(newX_data)
print(predict)

if(predict[0] == 0):
  print("Negative")
else:
  print("Positive")

0
[0]
Negative
